In [101]:
import dotenv

dotenv.load_dotenv()

True

In [102]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI

tools = [TavilySearchResults(max_results=1)]

chat = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0)

In [103]:
from langchain_community.tools.tavily_search import TavilySearchResults

tool = TavilySearchResults()

In [104]:
tool.invoke({"query": "support.madkudu.com/hc/en-us"})

[{'url': 'https://support.madkudu.com/hc/en-us/articles/10312876944653-Troubleshooting-MadKudu-Copilot',
  'content': 'If you are a MadKudu Admin, log in to app.madkudu.com and navigate to Settings > Users. Click "Add new use r", enter their email address and select the role "Sales user". They\'ll receive an invitation to accept and then will be able to connect in Salesforce. See more here: Invite users to MadKudu'},
 {'url': 'https://support.madkudu.com/hc/en-us/articles/360036535232-Introduction-to-MadKudu',
  'content': 'MadKudu Scoring. With MadKudu, qualify your prospects to send to Sales with AI scoring and directly in your CRM. MadKudu ingests as much data as possible to predict the likelihood of a prospect to convert into a customer. Get full control of your scoring in the Data Studio, with insights on your Ideal Customer Profile and most important actions ...'},
 {'url': 'https://support.madkudu.com/hc/en-us',
  'content': "A guide through MadKudu's features made for Operation

In [105]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("./madkudu.txt")
data = loader.load()

In [106]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

In [107]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

In [108]:
retriever = vectorstore.as_retriever(k=4)

docs = retriever.invoke("what is madkudu ?")

docs

[Document(page_content='How do I perform GDPR-compliant requests in MadKudu?\nRafikah Halim\nHow to make GDPR requests\nGet records deleted via CSV upload in MadKudu App\nGet records deleted via MadKudu GDPR deleted\nFor other types of requests', metadata={'source': './x.txt'}),
 Document(page_content='How do I perform GDPR-compliant requests in MadKudu?\nRafikah Halim\nHow to make GDPR requests\nGet records deleted via CSV upload in MadKudu App\nGet records deleted via MadKudu GDPR deleted\nFor other types of requests', metadata={'source': './madkudu.txt'}),
 Document(page_content='How do I perform GDPR-compliant requests in MadKudu?\nRafikah Halim\nHow to make GDPR requests\nGet records deleted via CSV upload in MadKudu App\nGet records deleted via MadKudu GDPR deleted\nFor other types of requests', metadata={'source': './madkudu.txt'}),
 Document(page_content='How do I perform GDPR-compliant requests in MadKudu?\nRafikah Halim\nHow to make GDPR requests\nGet records deleted via CSV 

In [109]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant working at Madkudu. You will need to chat and respond about question to help the user.",
        ),
        MessagesPlaceholder(variable_name="messages"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [110]:
from langchain.agents import AgentExecutor, create_openai_tools_agent

agent = create_openai_tools_agent(chat, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [124]:
from langchain_core.messages import HumanMessage

agent_executor.invoke({"messages": [HumanMessage(content="How do I perform GDPR-compliant requests in MadKudu?")]})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'How to perform GDPR-compliant requests in MadKudu'}`


[{'url': 'https://support.madkudu.com/hc/en-us', 'content': "A guide through MadKudu Copilot features for Sales. Admin & AI Scoring . A guide through MadKudu's features made for Operations-oriented roles. How To's. Explore MadKudu platform through Marketing and Sales use cases . Video Tutorials & Webinar Replays. Get the most of MadKudu learning content in interactive format. [Sign in access only ..."}]You can find information on how to perform GDPR-compliant requests in MadKudu by visiting the MadKudu support center at the following link: [MadKudu Support Center](https://support.madkudu.com/hc/en-us).

> Finished chain.


{'messages': [HumanMessage(content='How do I perform GDPR-compliant requests in MadKudu?')],
 'output': 'You can find information on how to perform GDPR-compliant requests in MadKudu by visiting the MadKudu support center at the following link: [MadKudu Support Center](https://support.madkudu.com/hc/en-us).'}

In [112]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

SYSTEM_TEMPLATE = """
Answer the user's questions based on the below context and any information you have about Madkudu.
If the context doesn't contain any relevant information to the question, don't make something up and just say "Im here to respond to madkudu question only":

<context>
{context}
</context>
"""

question_answering_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            SYSTEM_TEMPLATE,
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

document_chain = create_stuff_documents_chain(chat, question_answering_prompt)

In [113]:
from langchain_core.messages import HumanMessage

document_chain.invoke(
    {
        "context": docs,
        "messages": [
            HumanMessage(content="How do I perform GDPR-compliant requests in MadKudu?"),
        ],
    }
)

'To perform GDPR-compliant requests in MadKudu, you can follow these steps:\n\n1. Get records deleted via CSV upload in MadKudu App: You can upload a CSV file containing the records you want to be deleted to initiate the deletion process.\n\n2. Get records deleted via MadKudu GDPR deleted: You can use the specific feature or functionality within MadKudu designed for GDPR-related record deletion.\n\nFor other types of requests, you may need to refer to the specific documentation or support resources provided by MadKudu for guidance on how to make GDPR requests and ensure compliance with data protection regulations.'

In [123]:
document_chain.invoke(
    {
        "context": [],
        "messages": [
            HumanMessage(content="what is madkudu ?")
        ],
    }
)

'Madkudu is a predictive lead scoring and sales acceleration platform that helps businesses identify their best potential customers. It uses machine learning and predictive analytics to analyze customer data and provide insights into which leads are most likely to convert, allowing sales and marketing teams to prioritize their efforts effectively.'

In [121]:
from typing import Dict

from langchain_core.runnables import RunnablePassthrough


def parse_retriever_input(params: Dict):
    return params["messages"][-1].content


retrieval_chain = RunnablePassthrough.assign(
    context=parse_retriever_input | retriever,
).assign(
    answer=document_chain,
)